In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [3]:
def remove_background(video_file):
    frames = []
    _, reference_frame= video_file.read()
    reference_frame = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)
    video_file.set(cv2.CAP_PROP_POS_FRAMES, 0)
    retval = True
    while retval:
        retval, frame = video_file.read()
        if retval:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            frames.append(cv2.subtract(reference_frame, frame))
    
    return frames, reference_frame
        

In [7]:
# import videos
filepath = "C:/Users/rajor/OneDrive - Stanford/Experiments/Image processing/uF MPA/Videos/"
files = [file for file in os.listdir(filepath) if file.split('.')[1] == 'mp4']

In [8]:
video = cv2.VideoCapture(filepath + files[0])
frames, reference_frame = remove_background(video)

In [9]:
def compute_mean_px_sum(frame, idx, num_px=10):
    right = idx
    left = idx
    if idx < num_px:
        left = idx
        right = idx + num_px
    else:
        left = idx - num_px
        right = idx
    s = np.sum(frame[:, left:right], axis=0)
    return np.mean(s)

In [12]:
%matplotlib qt
displacement = []
num_frames = len(frames)
for num, frame in enumerate(frames):
    if np.sum(frame[:,0]) > 3000 and num % 250 == 0:
        sums = np.sum(frame.astype('float32'), axis=0)
        grad = np.diff(sums)
        d = np.argmin(sums) - np.argmin(grad)
        displacement.append(d)
        plt.subplot(2,1,1)
        plt.title(f'Frame number: {num}')
        plt.plot(sums, 'b-')
        plt.xlabel('x (in px)')
        plt.ylabel('Sum of pixel intensity')
        plt.subplot(2,1,2)
        plt.imshow(frame, cmap='gray', vmin=0, vmax=255)
        plt.xlabel('x (in px)')
        plt.ylabel('y (in px)')
        plt.pause(0.01)
        plt.clf()
#         break

# sums = np.sum(frames[20000], axis=0)
# plt.plot(sums)
# print(np.mean(frames[20000]))
# plt.imshow(frames[20000])
# plt.clf()
plt.show()

In [19]:
%matplotlib qt
plt.imshow(reference_frame.astype('float32'))
coords = plt.ginput(1)
plt.show()
print(coords[0][0])
# x_ref = 106

104.33870967741936


In [9]:
print(len(frames))
plt.imshow(frames[43000])
plt.show()

99437


In [10]:
t = np.arange(len(displacement))
plt.plot(t, np.array(displacement) * 20 / 26, 'b-')
plt.ylabel('Displacement (um)')
plt.xlabel('Time (sec)')
plt.show()

In [11]:
plt.subplot(2,1,1)
plt.show()

C:\Users\rajor\AppData\Local\Temp\ipykernel_24320\3634621176.py:1: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(2,1,1)


In [12]:
import pandas as pd
data= {'time':t[1:],
    'Displacement' :np.array(displacement[1:]) * 20 / 26}
df = pd.DataFrame(data)
df.head()

,time,Displacement


In [13]:
df.to_csv('uMPA_2.csv')